In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-30 05:33:41,758 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-30 05:33:41,759 - settings - Setting database.user to celiib
INFO - 2021-01-30 05:33:41,760 - settings - Setting database.password to newceliipass
INFO - 2021-01-30 05:33:42,303 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-30 05:33:42,304 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-30 05:33:42,320 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-30 05:33:42,610 - settings - Setting enable_python_native_blobs to True


In [2]:
#so that it will have the adapter defined
from datajoint_utils import *

In [3]:
test_mode = False

# Debugging the contains method

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-30 05:33:49,379 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-30 05:33:49,381 - settings - Setting database.user to celiib
INFO - 2021-01-30 05:33:49,382 - settings - Setting database.password to newceliipass
INFO - 2021-01-30 05:33:49,387 - settings - Setting enable_python_native_blobs to True


Sleeping 75 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-30 05:33:49,665 - settings - Setting enable_python_native_blobs to True


In [5]:
#(schema.jobs & "table_name='__decomposition'").delete()

# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 2217


In [10]:
decimation_version = 0
decimation_ratio = 0.25
# key_source = (minnie.Decimation().proj(decimation_version='version')  & 
#                   dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
#                   & minnie.MultiSomaProofread2() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid()).proj()))

key_source = ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0"))
                                                                  
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134219060901,0,0.25
864691134221889045,0,0.25
864691134242673681,0,0.25
864691134252908410,0,0.25
864691134316523453,0,0.25
864691134384033872,0,0.25
864691134391593458,0,0.25
864691134409056935,0,0.25
864691134428609977,0,0.25
864691134441691943,0,0.25


In [11]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25
process_version = 5

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    process_version : int unsigned #the version of the preprocessing pipeline run
    index : tinyint unsigned  #the index of the neuron object that resulted from that mesh (indexed starting at 0)
    ---
    multiplicity=null    : tinyint unsigned             # the number of somas found for this base segment
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """


    
    key_source = ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")) 
                                                                  
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)
        

        #3) Get the somas info 
        somas = du.get_soma_mesh_list(segment_id) 
        soma_ver = du.get_soma_mesh_list_ver(segment_id)
                  
        
        print(f"somas = {somas}")
                  
        #3b) Get the glia and nuclei information 
        glia_faces,nuclei_faces = du.get_segment_glia_nuclei_faces(segment_id,return_empty_list=True)
        
                  
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"],
        glia_faces=glia_faces,
        nuclei_faces = nuclei_faces,

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")

        #7) Pass stats and file location to insert
        new_key = dict(key,
                       ver = soma_ver,
                       process_version = process_version,
                       index = 0,
                       multiplicity=1,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [19]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table)#.delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

In [18]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

Populate Started



---- Working on Neuron 864691134793187861 ----
somas = [[<trimesh.Trimesh(vertices.shape=(9850, 3), faces.shape=(19564, 3))>], array([32.6147]), array([0.79])]

--- Beginning preprocessing of 864691134793187861---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(9850, 3), faces.shape=(19564, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(9850, 3), faces.shape=(19564, 3))>]
soma_mesh_list_centers = [array([1139753.95147208,  450670.86730965, 1094719.42081218])]
Getting Glia and Nuclei Pieces Subtracted Away 0.03258252143859863
 Splitting mesh after soma cancellation 0.10854959487915039
# of split_meshes = 4
 Containing Mesh Indices 0.0015060901641845703
 non_soma_touching_meshes 1.0251998901367188e-05
There were 3 pieces found after size threshold
 Finding inside pieces and no

somas = [[<trimesh.Trimesh(vertices.shape=(7477, 3), faces.shape=(14395, 3))>], array([20.2272]), array([0.441])]

--- Beginning preprocessing of 864691134834540726---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(7477, 3), faces.shape=(14395, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(7477, 3), faces.shape=(14395, 3))>]
soma_mesh_list_centers = [array([1175055.54767955, 1101922.16370202,  834215.76497258])]
Getting Glia and Nuclei Pieces Subtracted Away 0.027628421783447266
 Splitting mesh after soma cancellation 0.07578444480895996
# of split_meshes = 1
 Containing Mesh Indices 0.0012657642364501953
 non_soma_touching_meshes 8.106231689453125e-06
 Finding inside pieces and non_soma_touching meshes 9.5367431640625e-07

-----Before combining multiple mesh pieces-----
soma_containing_meshe

  0%|          | 0/1 [00:00<?, ?it/s]

new_floating_pieces = []
Total time for sig_non_soma_pieces= 0.04347968101501465
Total time for split= 1.6689300537109375e-06
Total time for mesh_pieces_connectivity= 0.15520548820495605
# of insignificant_limbs = 1 with trimesh : [<trimesh.Trimesh(vertices.shape=(816, 3), faces.shape=(1692, 3))>]
# of not_processed_soma_containing_meshes = 0 with trimesh : []


----- Working on Proper Limb # 0 ---------
meshparty_segment_size = 100
root_curr = [1175606.  1099536.   836933.9]
Time for preparing soma vertices and root: 0.0002148151397705078
invalidation_d = 12000
cc_vertex_thresh = 10


100%|██████████| 1/1 [00:00<00:00, 20.86it/s]


...


Removed 1 skeletal branches



2352 2353
For marked faces: None
empty_indices % = 0.0
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test



Total time for Skeletonization and Mesh Correspondence = 2.1763646602630615


 ----- Working on Stitching ----------
Total time for stitching floating pieces = 0.0022459030151367188
Total time for Concept Networks = 0.029110193252563477
Total time for all mesh and skeletonization decomp = 2.208000421524048
No soma volume ratios so computing them now
Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 2217 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_841956.off -o /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_841956_poisson.off -s /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/poisson_378955.mls
removed temporary input file: /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_841956.off
removed temporary output file: /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_841956_poisson.off
mesh.is

KeyboardInterrupt: 